# Assignment 8: Automated Machine Learning (Part 2)
## Objective:

As we learned from the class, the high demand for machine learning has produced a large amount of data scientists who have developed expertise in tools and algorithms. The features in the data will directly influence the results. However, it is tedious and unscalable to manually design and select features without domain knowledge. Thus, using some AutoML techniques will significantly help data scientists save labour and time. 
After completing this assignment, you should be able to answer the following questions:

1. Why do we need AutoML?
2. How does auto feature generation work?
3. How to use featuretools library to automatically generate features?
4. How to get useful features in a large feature space?

Imagine you are a data scientist in an online retailer company, for example, Amazon. Your task is to provide some recommendations to customers based on their historical purchase records.

In this assignment, we predict whether the customer will buy **Banana** in the next 4 weeks. It is a classification problem. To simplify the problem, we have already generated some features and provide the accuracy of the model (Random Forest model). The task for you is to generate **10** useful features and beat our model performance (AUC = 0.61, see below). 

For example, <br>
`MODE(orders.MODE(order_products.product_name)) = Bag of Organic Bananas` means whether the most frequent purchase of the customer is Bag of Organic Bananas. 

```
1: Feature: MODE(orders.MODE(order_products.product_name)) = Bag of Organic Bananas
2: Feature: MODE(order_products.aisle_id) is unknown
3: Feature: SUM(orders.NUM_UNIQUE(order_products.product_name))
4: Feature: MODE(orders.MODE(order_products.product_name)) = Boneless Skinless Chicken Breasts
5: Feature: MODE(order_products.product_name) = Boneless Skinless Chicken Breasts
6: Feature: STD(orders.NUM_UNIQUE(order_products.aisle_id))
7: Feature: MODE(order_products.aisle_id) = 83
8: Feature: MEDIAN(orders.MINUTE(order_time))
9: Feature: MODE(orders.DAY(order_time)) = 23
10: Feature: MODE(orders.MODE(order_products.department)) = produce

AUC 0.61
```


## Preliminary
If you never use featuretools before, you need to learn some basic knowledge of this topic. 
I found that these are some good resources: 
* [featuretools documentation](https://docs.featuretools.com/en/stable/)
* [Tutorial: Automated Feature Engineering in Python](https://towardsdatascience.com/automated-feature-engineering-in-python-99baf11cc219)

The data can be downloaded from [A8-2-data.zip](A8-2-data.zip). 

## 0. Preparation
Import relevant libraries and load the dataset: <br>
users: <br>
* user_id: customer identifier
* label:  1 if the customer will buy banana in next 4 weeks, 0 otherwise

orders: <br>
* order_id: order identifier
* user_id: customer identifier
* order_time: date of the order was placed on 

order_products: <br>
* order_id: order identifier
* order_product_id: foreign key
* reordered:  1 if this product has been ordered by this user in the past, 0 otherwise
* product_name: name of the product
* aisle_id: aisle identifier
* department: the name of the department
* order_time: date of the order was placed on

In [1]:
import pandas as pd

import featuretools as ft
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import os
ft.__version__

# list all rows and columns
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
orders = pd.read_csv("orders.csv")
order_products = pd.read_csv("order_products.csv")
users = pd.read_csv("users.csv")

# could drop columns here, if needed
orders.drop(orders.columns[0], axis=1, inplace=True)
order_products.drop(order_products.columns[0], axis=1, inplace=True)
users.drop(users.columns[0], axis=1, inplace=True)

print(users["label"].value_counts())
print(orders.shape, order_products.shape)
users.head()

False    628
True     139
Name: label, dtype: int64
(5997, 3) (57780, 7)


,user_id,label
0,1,False
1,2,True
2,3,False
3,7,False
4,10,False


## Task 1. Feature Generation
In this task, you need to use featuretools to generate candidate features by using the above three tables.

### 1.1 Representing Data with EntitySet

Define entities and their relationships (see [https://docs.featuretools.com/en/stable/generated/featuretools.EntitySet.html](https://docs.featuretools.com/en/stable/generated/featuretools.EntitySet.html))

In [4]:
# Get the relationship between entities
def load_entityset(orders, order_products, users):
    # --- Write your code below ---
    # return the EntitySet object
    dataframes = {
        'orders': (orders, 'order_id', 'order_time'),
        'order_products': (order_products, 'order_product_id', 'order_time'),
        'users': (users, 'user_id')
    }
    
    relationships = [
            ("orders", "order_id", "order_products", "order_id"),
            ("users", "user_id", "orders", "user_id" ),
    ]

    return ft.EntitySet("entity-set", dataframes, relationships)

### 1.2 Deep Feature Synthesis

In [5]:
# Automatically generate features
es = load_entityset(orders, order_products, users)

# use ft.dfs to perform feature engineering
# --- Write your code below ---
feature_matrix, feature_defs = ft.dfs(entityset=es, target_dataframe_name="users")

In [6]:
# output what features you generate
feature_matrix

,label,COUNT(orders),COUNT(order_products),MAX(order_products.aisle_id),MAX(order_products.reordered),MEAN(order_products.aisle_id),MEAN(order_products.reordered),MIN(order_products.aisle_id),MIN(order_products.reordered),MODE(order_products.department),MODE(order_products.product_name),NUM_UNIQUE(order_products.department),NUM_UNIQUE(order_products.product_name),SKEW(order_products.aisle_id),SKEW(order_products.reordered),STD(order_products.aisle_id),STD(order_products.reordered),SUM(order_products.aisle_id),SUM(order_products.reordered),MAX(orders.COUNT(order_products)),MAX(orders.MEAN(order_products.aisle_id)),MAX(orders.MEAN(order_products.reordered)),MAX(orders.MIN(order_products.aisle_id)),MAX(orders.MIN(order_products.reordered)),MAX(orders.NUM_UNIQUE(order_products.department)),MAX(orders.NUM_UNIQUE(order_products.product_name)),MAX(orders.SKEW(order_products.aisle_id)),MAX(orders.SKEW(order_products.reordered)),MAX(orders.STD(order_products.aisle_id)),MAX(orders.STD(order_products.reordered)),MAX(orders.SUM(order_products.aisle_id)),MAX(orders.SUM(order_products.reordered)),MEAN(orders.COUNT(order_products)),MEAN(orders.MAX(order_products.aisle_id)),MEAN(orders.MAX(order_products.reordered)),MEAN(orders.MEAN(order_products.aisle_id)),MEAN(orders.MEAN(order_products.reordered)),MEAN(orders.MIN(order_products.aisle_id)),MEAN(orders.MIN(order_products.reordered)),MEAN(orders.NUM_UNIQUE(order_products.department)),MEAN(orders.NUM_UNIQUE(order_products.product_name)),MEAN(orders.SKEW(order_products.aisle_id)),MEAN(orders.SKEW(order_products.reordered)),MEAN(orders.STD(order_products.aisle_id)),MEAN(orders.STD(order_products.reordered)),MEAN(orders.SUM(order_products.aisle_id)),MEAN(orders.SUM(order_products.reordered)),MIN(orders.COUNT(order_products)),MIN(orders.MAX(order_products.aisle_id)),MIN(orders.MAX(order_products.reordered)),MIN(orders.MEAN(order_products.aisle_id)),MIN(orders.MEAN(order_products.reordered)),MIN(orders.NUM_UNIQUE(order_products.department)),MIN(orders.NUM_UNIQUE(order_products.product_name)),MIN(orders.SKEW(order_products.aisle_id)),MIN(orders.SKEW(order_products.reordered)),MIN(orders.STD(order_products.aisle_id)),MIN(orders.STD(order_products.reordered)),MIN(orders.SUM(order_products.aisle_id)),MIN(orders.SUM(order_products.reordered)),MODE(orders.DAY(order_time)),MODE(orders.MODE(order_products.department)),MODE(orders.MODE(order_products.product_name)),MODE(orders.MONTH(order_time)),MODE(orders.WEEKDAY(order_time)),MODE(orders.YEAR(order_time)),NUM_UNIQUE(orders.DAY(order_time)),NUM_UNIQUE(orders.MODE(order_products.department)),NUM_UNIQUE(orders.MODE(order_products.product_name)),NUM_UNIQUE(orders.MONTH(order_time)),NUM_UNIQUE(orders.WEEKDAY(order_time)),NUM_UNIQUE(orders.YEAR(order_time)),SKEW(orders.COUNT(order_products)),SKEW(orders.MAX(order_products.aisle_id)),SKEW(orders.MAX(order_products.reordered)),SKEW(orders.MEAN(order_products.aisle_id)),SKEW(orders.MEAN(order_products.reordered)),SKEW(orders.MIN(order_products.aisle_id)),SKEW(orders.MIN(order_products.reordered)),SKEW(orders.NUM_UNIQUE(order_products.department)),SKEW(orders.NUM_UNIQUE(order_products.product_name)),SKEW(orders.STD(order_products.aisle_id)),SKEW(orders.STD(order_products.reordered)),SKEW(orders.SUM(order_products.aisle_id)),SKEW(orders.SUM(order_products.reordered)),STD(orders.COUNT(order_products)),STD(orders.MAX(order_products.aisle_id)),STD(orders.MAX(order_products.reordered)),STD(orders.MEAN(order_products.aisle_id)),STD(orders.MEAN(order_products.reordered)),STD(orders.MIN(order_products.aisle_id)),STD(orders.MIN(order_products.reordered)),STD(orders.NUM_UNIQUE(order_products.department)),STD(orders.NUM_UNIQUE(order_products.product_name)),STD(orders.SKEW(order_products.aisle_id)),STD(orders.SKEW(order_products.reordered)),STD(orders.SUM(order_products.aisle_id)),STD(orders.SUM(order_products.reordered)),SUM(orders.MAX(order_products.aisle_id)),SUM(orders.MAX(order_products.reordered)),SUM(orders.MEAN(order_

## Task 2. Feature Selection
In this task, you are going to select 10 features that are useful and train the *Random Forest* model. The goal is to beat the accuracy performance as we have shown before. Note that you have to use the Random Forest and the hyperparameters we provide in Section 2.2. In other words, your job is to achieve a higher AUC than 0.61 through feature generation/selection rather than through hyperparameter tuning or model selectoin. 

### 2.1 Select top features

In [7]:
# --- Write your code below ---
# Select top-10 features and return X, y (X.shape = (767, 10)
categorical_cols = feature_matrix.select_dtypes(include=['category']).columns

# Select the top-10 most correlated numerical features
df = feature_matrix.drop(columns=categorical_cols)
corr = df.corr(method='spearman').iloc[0]
features_selected = corr.abs().sort_values(ascending=False)[1:11].index.to_list()

X = feature_matrix[features_selected]
y = feature_matrix.iloc[:, 0]

print(X.shape)

(767, 10)


### 2.2 Get accuracy and list features

In [8]:
clf = RandomForestClassifier(n_estimators=400, n_jobs=-1)
scores = cross_val_score(estimator=clf,X=X, y=y, cv=3,
                             scoring="roc_auc", verbose=True)

print("AUC %.2f" % (scores.mean()))

# Print top-10 features
print('--- Top 10 features selected ---')
i = 1
for feature in X.columns:
    print(i, feature)
    i += 1

AUC 0.66
--- Top 10 features selected ---
1 MEAN(orders.NUM_UNIQUE(order_products.product_name))
2 MEAN(orders.COUNT(order_products))
3 MEAN(orders.SUM(order_products.aisle_id))
4 COUNT(order_products)
5 SUM(orders.NUM_UNIQUE(order_products.product_name))
6 SUM(order_products.aisle_id)
7 NUM_UNIQUE(order_products.product_name)
8 MIN(orders.COUNT(order_products))
9 MIN(orders.NUM_UNIQUE(order_products.product_name))
10 MEAN(orders.NUM_UNIQUE(order_products.department))


## Task3. Writing Questions

1. Please list three advantages and disadvantages of featuretools. 
2. For those disadvantages you listed above, do you have any idea to improve it? 

--- Write your answer here---
#### 1
**Advantages of featuretools**
1. It automates the process of feature generation, which can save a significant amount of time and effort compared to manual feature engineering. Id can generate new features by automatically applying mathematical operations and aggregations across different related tables.
2. Scalability: It is designed to efficiently handle large datasets and complex data schemas, allowing for the generation of features at scale.
3. It can be easily integrated with other Python data science and machine learning libraries, enhancing the overall workflow.

**Disadvantages of featuretools**
1. Overfitting Risk: Automated generation of a large number of features can lead to overfitting, especially if not paired with appropriate feature selection or regularization techniques.
2. Resource Intensive: Generating a vast number of features, especially on large datasets, can be computationally intensive and may require significant memory and processing power.
3. Quality of Generated Features: Not all automatically generated features are useful or relevant. Identifying and selecting the most impactful features still requires domain knowledge and critical analysis.


#### 2 Improvement for the disadvantages
- Overfitting: Feature selection helps in reducing the feature space to the most informative ones, thereby minimizing complexity and the risk of overfitting. Regularization methods, such as Lasso or Ridge regression in linear models or built-in features in tree-based models, penalize the less important features to prevent overfitting. Cross-validation ensures the model's ability to generalize by testing it on unseen data, providing a more accurate assessment of its performance.

- Resource Intensive: The challenge can be mitigated by adopting an incremental approach to feature generation, utilizing distributed computing resources, and optimizing data types. Incrementally generating and evaluating features allows for the early identification of valuable features without the need for extensive computation. Distributed computing frameworks can efficiently handle large datasets and intensive computations. Optimizing data types for memory efficiency can significantly reduce the computational resources required, making the process more manageable.

- Quality of Generated Features: Working with domain experts to guide the feature generation process ensures the relevance of the features. Developing a set of best practices based on historical successes can guide the generation of more meaningful features. A thorough evaluation of each feature's impact on the model's performance helps in identifying and retaining only those that provide significant value, thereby enhancing the overall quality of the feature set.

## Submission
Complete the code in this notebook, and submit it to the CourSys activity Assignment 8.